## 1.多层感知机从零实现

In [ ]:
#mlp-scratch ------多层感知机从零实现
import torch
from torch import nn
from d2l import torch as d2l

#数据导入
batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

#初始化模型参数
num_inputs,num_outputs,num_hiddens=784,10,256 #输入，输出，隐藏层隐藏单元
W1=nn.Parameter(torch.randn(num_inputs,num_hiddens,requires_grad=True)*0.01) #输入层到隐藏层
b1=nn.Parameter(torch.zeros(num_hiddens,require_grad=True))
W2=nn.Parameter(torch.randn(num_inputs,num_hiddens,require_grad=True)*0.01) #隐藏层到输出层
b2=nn.Parameter(torch.zeros(num_outputs,require_grad=True))

params=[W1,b1,W2,b2]

#激活函数：实现ReLU函数
def relu(X):
    a=torch.zeros_like(X)
    return torch.max(X,a)

#定义模型
def net(X):
    X=X.reshape((-1,num_inputs)) #将每个二维图像转换为一个长度为num_inputs的向量
    H=relu(X@W1+b1) #这里@表示矩阵乘法
    return (H@W2+b2)

#损失函数
loss=nn.CrossEntropyLoss() #使用内置API计算交叉熵损失

#训练：与softmax训练过程完全相同
num_epochs,lr=10,0.1
updater=torch.optim.SGD(params,lr=lr)
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,updater)

#模型应用于测试数据
d2l.predict_ch3(net,test_iter)

## 2.多层感知机简洁实现--使用API构建网络

In [ ]:
#mlp-concise ------多层感知机简洁实现--使用API构建网络
import torch
feom torch import nn
from d2l import torch as d2l

#模型
net=nn.Sequential(nn.Flatten(),
                 nn.Linear(784,256),
                 nn.ReLU().
                 nn.Linear(256,10))
def init_weights(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,std=0.01)
net.apply(init_weights);

#交叉熵损失函数
loss=nn.CrossEntropyLoss() 

#训练
batch_size,lr,num_epochs=256,0.1,10
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size) #数据导入
trainer=torch.optim.SGD(net.parameters(),lr=lr) #参数优化
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,trainer)

## 3.多项式拟合--模型复杂性

In [ ]:
import math 
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

#生成数据
max_degree=20 #多项式最大阶数
n_train,n_test=100,100 #训练集和测试集大小
true_w=np.zeros(max_degree) #分配空间
true_w[0:4]=np.array([5,1.2,-3.4,5.6])

features=np.random.normal(size=(n_train+ntest,1)) #训练集和测试集均为100x1的正态分布随机数
np.random.shuffle(features)
poly_features=np.power(features,np.arange(max_degree).reshape(1,-1)) #feature中的数，通过广播机制求每个数的0~19次方，ploy_features.shape=200x20。np.arange(max_degree).reshape(1, -1)将轴数从1调整为2
for i in range(max_degree):
    poly_features[:,i]/=math.gamma(i,1) #自变量的n次方除以对应阶数n的gamma(n)=(n-1)!
labels=np.dot(poly_features,true_w) #各阶数的自变量乘以系数得到输出y。labels维度：(n_train+n_test,)
labels+=np.random.normal(scale=0.1,size=labels.shape) #加上噪声
true_w,features,poly_features,labels=[torch.tensor(x,dtype=torch.float32) 
                                     for x in [true_w,features,poly_features,labels]] #numpy ndarray转换为tensor

#损失评估----评估给定数据集上模型的
def evaluate_loss(net,data_iter,loss): #@save  #输入网络类型，数据集，损失函数
    metric=d2l.Accumulator(2) #构建含2个0元素的列表，第1个元素表示损失总和，第2个元素表示元素数目。 实用程序类Accumulator，用于对多个变量进行累加
    for X,y in data_iter:
        out=net(X) #训练得到预测输出
        y=y.reshape(out.shape)
        l=loss(out,y) #损失函数计算损失
        metrix.add(l.sum(),l.numel()) #numel()返回数组中元素的个数
    return metric[0]/metric[1] #返回训练样本的损失均值

#训练函数
def train(train_features,test_features,train_labels,test_labels,
         num_epochs=400):
    
    #损失函数
    loss=nn.MSELoss() #使用均方差损失函数MSELoss
    
    #网络模型
    input_shape=train_features.shape(-1) #shape[-1]获取shape中的最后一个元素，即样本数量
    net=nn.Sequential(nn.Linear(input_shape,1,bias=False)) ## 不设置偏置，因为已经在多项式特征中实现了偏置
    
    #构建训练/测试数据集
    batch_size=min(10,train_labels.shape[0]) #获取批量大小
    train_iter=d2l.load_array((train_features,train_labels.shape(-1,1),batch_size)) #加载训练数据集
    test_iter=d2l.load_array((test_features,test_labels.reshape(-1,1),batch_size,is_train=False))#加载测试数据集
    
    #优化函数
    trainer=torch.opti.SGD(net.parameters(),lr=0.01) #使用随机梯度下降算法更新参数
    
    #绘制画布、坐标轴、图例
    animator=d2l.Animator(xlabel='epoch',ylabel='loss',yscale='log',xlim=[1,num_epochs],ylim=[le-3,le2],legend=['train','test'])#调用d2l.Animator(),绘制训练过程中训练集和测试集上的epoch-loss图
    
    #训练
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net,train_iter,loss,trainer) #调用训练函数进行训练，不断更新参数--多项式系数
        if epoch==0 or (epoch+1)%20==0: #由训练进度绘制数据点，分别绘制训练集和测试集平均损失。evaluate_loss()计算数据集上的损失均值
            animator.add(epoch+1,(evaluate_loss(net,train_iter,loss),evaluate_loss(net,test_iter,loss)))
    print('weight:',net[0].weight.data.numpy()) #打印多项式系数
    
#三阶多项式函数拟合
train(poly_features[:n_train,:4],poly_features[n_train:,:4],labels[:n_train],labels[n_train:]) #poly_features[:ntrain,:4] 获取前ntrain个样本的前4个特征维度，最高为3阶，即 1, x, x^2/2!, x^3/3!。已定义n_train=100，前100个样本作为训练集，剩余100个样本最为测试集
    
#线性函数拟合(欠拟合)
train(poly_features[:n_train,:2],poly_features[n_train:,:2],labels[:n_train],labels[n_train:]) #只选择前两个特征，最高阶为1，即1, x

#高阶多项式拟合(过拟合)
train(poly_features[:n_train,:],poly_features[n_train:,:],labels[:n_train],labels[n_train:]) #选择所有特征，即包含最高阶数19


## 4.权重衰减

In [1]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l

#生成训练集和测试集
n_train,n_test,num_inputs,batch_size=20,100,200,5 #训练集样本数20，测试集样本数100，样本特征维度200，小批量样本数5
true_w,true_b=torch.ones((num_inputs,1))*0.01,0.05 #w,b的真实参数值，b为0.05
train_data=d2l.synthetic_data(true_w,true_b,n_train) #调用synthetic_data()合成数据集 y = Xw + b + noise，返回输入样本矩阵和labels列向量
train_iter=d2l.load_array(train_data,batch_size) #从训练集获取小批量样本
test_data=d2l.synthetic_data(true_w,true_b,n_test) #生成测试数据集 y = Xw + b + noise
test_iter=d2l.load_array(test_data,batch_size,is_train=False) #从测试集获取小批量样本

#初始化模型参数
def init_params(): #随机初始化模型参数
    w=torch.normal(0,1,size=(num_inputs,1),require_grad=True) #从一个标准正态分布N~(0,1)中提取数据，形成一个(num_inputs,1)的权重w矩阵，设置需要求导计算
    b=torch.zeros(1,require_grad=True) #偏置b设置为包含1个0元素的Tensor张量
    return [w,b]

#定义L2范数惩罚
def l2_penalty(w):
    return torch.sum(w.pow(2))/2 #将w所有元素平方后再求和

#权重衰减训练函数---从0实现不使用框架
def train(lambd): #lambd为非负超参数，用于平衡L2惩罚的程度
    #初始化、定义网络模型、定义损失函数
    w,b=init_params() #调用函数将参数初始化
    net,loss=lambda X:d2l.linreg(X,w,b),d2l.squared_loss #定义网络net和损失函数loss。net采用线性网络matmul(X,w)+b，loss为平方损失。net为lambd表达式，loss为d2l.squared_loss
    num_epochs,lr=100,0.003 #设置训练迭代的次数和梯度下降步长默认值
    
    #设置画布
    animator=d2l.Animator(xlabel='epochs',ylabel='loss',yscale='log',xlim=[5,num_epochs],legend=['train','test']) #调用d2l.Animator(),绘制训练过程中训练集和测试集上的epoch-loss图
    
    #训练模型
    for epoch in range(num_epochs): 
        for X,y in train_iter:
            l=loss(net(X),y)+lambd*l2penalty(w) #原来的损失函数中加入L2范数惩罚项。广播机制使l2_penalty(w)成为一个长度为batch_size的向量
            l.sum().backward() #反向传播计算变量梯度
            d2l.sgd([w,b],lr,batch_size) #采用小批量随机梯度下降优化算法，对模型参数w,b进行更新优化
        if (epoch+1)%5==0:
            animator.add(epoch+1,(d2l.evaluate_loss(net,train_iter,loss),d2l.evaluate_loss(net,test_iter,loss))) #计算训练损失和测试误差，并绘制出数据点
    
    print('w的L2范数：'torch.norm(w).item()) #torch.num()是计算输入w的范数，默认二范数。pytorch中的item()是得到一个元素张量中的元素值，会将一个0维张量转换成浮点数
    
#忽略正则化直接计算
train(lambd=0)#lambd=0禁用权重衰减，会出现严重过拟合

#使用权重衰减
train(lambd=3) 


#权重衰减训练函数---简洁实现:使用pytorch框架
def train_concise(wd):
    #初始化、定义网络模型、定义损失函数
    net=nn.Sequential(nn.Linear(num_inputs,1))
    for param in net.parameters():
        param.data.normal_()
    loss=nn.MSELoss()
    num_epochs,lr=100,0.003
    
    #设置优化函数
    trainer=torch.optim.SGD([
        {"params":net[0].weight,'weight_decay':wd},
        {"params":net[0].bias}],lr=lr) # 调用pytorch中的优化算法，在优化算法中将系数w设置为衰减，偏置参数没有衰减
    
    #设置画布
    animator=d2l.Animator(xlabel='epochs',ylabel='loss',yscale='log',xlim=[5,num_epochs],legend=['train','test'])
    
    #模型参数训练
    for epoch in range(num_epochs):
        for X,y in trian_iter:
            trainer.zero_grad() #将模型参数的梯度值清0
            l=loss(net(X),y)
            l.backward()
            trainer.step()
        if (epoch+1)%5==0:
            animator.add(epoch+1.(d2l.evaluate_loss(net,train_iter,loss),d2l.evaluate_loss(net,test_iter,loss)))
        
    print('w的L2范数：'，net[0].weight.norm().item()) #输出w的L2范数
    
#忽略正则化直接计算
train(lambd=0)#lambd=0禁用权重衰减，会出现严重过拟合

#使用权重衰减
train(lambd=3) 

SyntaxError: invalid syntax (3676027538.py, line 43)

## 5.暂退法 Dropout

In [ ]:
### 从0实现
import torch
from torch import nn
from d2l import torch as d2l

#实现dropout算法
def dropout_layer(X,dropout):
    assert 0<=dropout<=1 #dropout为隐藏单元丢弃概率。需要概率取值在[0,1]。#assert（断言）用于判断一个表达式，在表达式条件为 false 的时候触发异常
    if dropout==1: #doprout为1，所有元素军被丢弃(置0)
        return torch.zeros_like(X)
    if dropout==0: #dropout为0，所有元素保留不做修改
        return X
    mask=(torch.rand(X.shape)>dropout).float() #随机设置被丢弃的隐藏层单元。torch.rand返回了一个X.shape的张量，元素为从0-1的均匀分布中抽取的一组随机数。随机数与dropout比较之后的True、False列表，通过float()转换为0/1列表
    return mask*X/(1.0-dropout) #dropout表达式，修改输入X中的元素

#测试dropout函数
X=torch.arange(16,dtype=torch.float32).reshape((2,8))
print(X)
print(dropout_layer(X,0.))
print(dropout_layer(X,0.5)) #丢弃概率0.5
print(dropout_layer(X,1))

#定义模型参数
#使用Fashion-MNIST数据集；定义具有两个隐藏层的多层感知机，每个隐藏层包含256个单元
num_inputs,num_outputs,num_hiddens1,num_hiddens2=784,10,256,256

#定义模型
#dropout应用于每个隐藏层的输出（在激活函数之后）， 并且可以为每一层分别设置暂退概率；通常在靠近输入层的地方设置较低的暂退概率
dropout1,dropout2=0.2,0.5
class Net(nn.Module):
    def __init__(self,num_inputs,num_outputs,num_hiddens1,num_hiddens2,is_training=True):
        super(Net,self).__init__()
        self.num_inputs=num_inputs
        self.training=is_training
        self.lin1=nn.Linear(num_inputs,num_hiddens1)
        self.lin2=nn.Linear(num_hiddens1,num_hiddens2)
        self.lin3=nn.Linear(num_hiddens2,num_outputs)
        self.relu=nn.ReLU()
        
    def forward(self,X):
        #输入层--第一隐藏层
        H1=self.relu(self.lin1(X.reshape((-1,self.num_inputs))))
        if self.training==True:
            H1=dropout_layer(H1,dropout1)
            
        #第一隐藏层--第二隐藏层
        H2=self.relu(self.lin2(H1))
        if self.training==True:
            H2=dropout_layer(H2,dropout2)
            
        #第二隐藏层--输出层
        out=self.lin3(H2)
        
        return out

#Net类实例化
net=Net(num_inputs,num_outputs,num_hiddens1,num_hiddens2,is_training=True)

#训练和测试
num_epochs,lr,batch_size=10,0.5,256 #初始化迭代次数，梯度下降步长，样本批量大小
loss=nn.CrossEntropyLoss() #使用交叉熵损失函数
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size) #加载训练集、测试集
trainer=torch.optim.SGD(net.parameter(),lr=lr) #使用小批量随机梯度下降算法对参数进行优化
d2l.train_ch3(net,train_iter,test_iter,loss,,num_epochs,trainer) #模型训练调用已有函数；网络模型为NET实例net


### 简洁实现---使用框架
net=nn.Sequential(nn.Flatten(),
                 #输入--hidden1
                 nn.Linear(784,256),
                 nn.ReLU(),
                 nn.Dropout(dropout1),
                
                 #hidden1--hidden2
                 nn.Linear(256,256),
                 nn.ReLU(),
                 nn.Dropout(dropout2)
                
                 #hidden2--out
                 nn.Linear(256,10))
                 
def init_weights(m): #此函数用于初始化线性模型的权重参数
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight,std=0.01) #从均值0和标准差0.01正态分布中生成值，填充输入的张量m.weight
        
net.apply(init_weights) #网络模型参数初始化

trainer=torch.optim.SGD(net.parameters(),lr=lr)
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochstrain)
